In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from bwsi_grader.cogworks.autograd import grade_op, grade_arithmetic
from bwsi_grader.cogworks.autograd import grade_backprop, grade_op_backprop
from operations import Add, Subtract, Multiply, Divide, Power

In [ ]:
class Number(object):
    def __repr__(self):
        return "Number({})".format(self.data)

    def __init__(self, obj, *, creator=None):
        assert isinstance(obj, (Number, int, float, np.generic))
        self.data = obj.data if isinstance(obj, Number) else obj
        self._creator = creator
        self.grad = None

    @property
    def creator(self):
        return self._creator
    
    @staticmethod
    def _op(Op, a, b):
        if not isinstance(a, Number):
            a = Number(a)
        if not isinstance(b, Number):
            b = Number(b)
            
        result = f(a, b)
        return Number(result, creator=f)

    def __add__(self, other):
        return self._op(Add, self, other)

    def __radd__(self, other):
        return self._op(Add, other, self)

    def __mul__(self, other):
        return self._op(Multiply, self, other)

    def __rmul__(self, other):
        return self._op(Multiply, other, self)

    def __truediv__(self, other):
        return self._op(Divide, self, other)

    def __rtruediv__(self, other):
        return self._op(Divide, other, self)

    def __sub__(self, other):
        return self._op(Subtract, self, other)

    def __rsub__(self, other):
        return self._op(Subtract, other, self)

    def __pow__(self, other):
        return self._op(Power, self, other)

    def __rpow__(self, other):
        return self._op(Power, other, self)

    def __neg__(self):
        return -1*self
    
    def __eq__(self, value):
        if isinstance(value, Number):
            value = value.data
        return self.data == value

    def backprop(self, grad=1):
        if self.grad is None:
            self.grad = grad
        else:
            self.grad += grad
        
        if self._creator is not None:
            self._creator.backprop(grad = grad)
    
    def null_gradients(self):
        self.grad = None
        if self._creator is not None:
            self._creator.null_gradients()
